# COMP0036: Group Coursework 

## Beat the Bookie

`<insert Introduction here>`

### Package Import

In [7]:
#Standard Python libraries for data and visualisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Import models
from sklearn.ensemble import RandomForestRegressor

#Import error metric
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#Import data munging tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

#Display charts in the notebook
%matplotlib inline

### Data Import
``<insert a brief description of our datasets here>``

### Data Transformation and Exploration
``<insert a brief description of our preperation and standardisation process here>``

### Methodology Overview
``Pipeline overview``

### Model training and validation

#### Shots xG model

#### Non-shots xG model


#### ELO Rating classifier

In [ ]:
class GoalElo:
    def __init__(self, initial_rating=0, learning_rate=0.05, draw_size=0.5):
        self.offensive_ratings = defaultdict(lambda: initial_rating)
        self.defensive_ratings = defaultdict(lambda: initial_rating)
        self.match_count = defaultdict(lambda: 0)
        self.learning_rate = learning_rate
        self.draw_size = draw_size

    def predict(self, team, opponent):
        ''' Predicts the number of goals team will score against opponent. '''
        return self.offensive_ratings[team] - self.defensive_ratings[opponent]

    def predict_result(self, team, opponent):
        ''' Predicts the result of a match. 1 if team wins, 0 if opponent wins and 0.5 if it is a draw. '''
        goals_scored = self.predict(team, opponent)
        goals_conceded = self.predict(opponent, team)
        return self.classify_result(goals_scored, goals_conceded)

    def classify_result(self, goals_scored, goals_conceded):
        goal_difference = goals_scored - goals_conceded
        # result = round(1 / (1 + 10**(-goal_difference)))
        result = 1 if goal_difference > 0 else 0
        if abs(goal_difference) < self.draw_size:
            result = 0.5
        return result


    def predict_data(self, df):
        ''' Takes a data frame of home and away teams to predict the number of goals and result of '''
        out = df.copy()
        for i, row in out.iterrows():
            out.at[i, 'EHG'] = self.predict(row['HomeTeam'], row['AwayTeam'])
            out.at[i, 'EAG'] = self.predict(row['AwayTeam'], row['HomeTeam'])
            out.at[i, 'ER'] = decode_result(self.predict_result(row['HomeTeam'], row['AwayTeam']))
        return out

    def update_match(self, home, away, home_actual_goals, away_actual_goals):
        ''' Updates the offensive and defensive ratings of both teams in a match. '''
        home_expected_goals = self.predict(home, away)
        away_expected_goals = self.predict(away, home)
        self.offensive_ratings[home] += self.learning_rate * (home_actual_goals - home_expected_goals)
        self.offensive_ratings[away] += self.learning_rate * (away_actual_goals - away_expected_goals)
        self.defensive_ratings[home] += self.learning_rate * (away_expected_goals - away_actual_goals)
        self.defensive_ratings[away] += self.learning_rate * (home_expected_goals - home_actual_goals)
        self.match_count[home] += 1
        self.match_count[away] += 1

    def ratings_dataframe(self):
        ''' Creates an easy to read dataframe of the ratings '''
        df = pd.DataFrame(self.offensive_ratings.items(), columns=['Team', 'Offensive Rating'])
        df['Defensive Rating'] = df['Team'].map(self.defensive_ratings)
        df['Matches'] = df['Team'].map(self.match_count)
        df = df.sort_values('Offensive Rating', ascending=False)
        return df

    def fit(self, df):
        ''' Takes a data frame of matches with columns HomeTeam, AwayTeam, Predicted FTHG, Predicted FTAG and updates teams ratings using the data in order. '''
        for i, row in df.iterrows():
            if 'Predicted FTHG' in row:
                self.update_match(row['HomeTeam'], row['AwayTeam'], row['Predicted FTHG'], row['Predicted FTAG'])
            else:
                self.update_match(row['HomeTeam'], row['AwayTeam'], row['FTHG'], row['FTAG'])


In [ ]:
# Train elo ratings
goal_elo = GoalElo()
goal_elo.fit(training)


In [ ]:
# Predict number of goals for use in training classifier
goal_predicted_data = goal_elo.predict_data(training)

X = np.array([data['EHG'].to_numpy(), data['EAG'].to_numpy()]).T
y = data['FTR'].to_numpy()
# y = [decode_result(r) for r in data['FTR'].to_numpy()]

goal_result_classifier = SVC(gamma='auto')
goal_result_classifier.fit(X, y)

In [ ]:
goal_prediction = goal_elo.predict_data(test)
X_test = np.array([goal_prediction['EHG'].to_numpy(), goal_prediction['EAG'].to_numpy()]).T
y_test = goal_prediction['FTR'].to_numpy()
y_pred = model.predict(X_test)
y_pred2 = goal_prediction['ER'].to_numpy()

print("Accuracy:")
print("DS: ", accuracy_score(y_test, y_pred2))
print("SVC: ", accuracy_score(y_test, y_pred))

print("F1 Score:")
print("DS: ", f1_score(y_test, y_pred2, average='weighted'))
print("SVC: ", f1_score(y_test, y_pred, average='weighted'))

### Results

``<insert model tests here>``

### Final predictions